In [ ]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [ ]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-07-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-07-31", "%Y-%m-%d")

#Create a range of dates to iterate 
#Note: Periods here represents the number of days it will create from the start date 
#We also do a +2 since we want to get data from the last day  
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#parameters
sort_type="num_comments"
sort="desc"
fields=["title","author","subreddit","score","selftext","num_comments","created_utc"]
subreddit = 'NBA'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#looping through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        time.sleep(1)

In [ ]:
to_readable_date(1596315600)

In [ ]:
flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#storing list into the data frame
df = pd.DataFrame.from_dict(flat_list)
# convert epoc to datetime
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
display(df)

In [ ]:
#total submissions
total_rows = df['title'].count()
total_rows

In [180]:
#max submissions of author
total_subm =df['author'].value_counts(sort=True).head(1)
#max submissions of author
total_score= df[['title','score']]
#max num comments
total_comms= df[['title','num_comments']]

In [181]:
# item No.4 questions

print('How many submissions were you able to gather?:\n', total_rows)

print('\nWho has the most submissions?:\n', total_subm)

print('\nWhich submission has the highest score?:\n ',total_score[total_score['score']==total_score['score'].max()])

print('\nWhich submission has the highest number of comments?:\n ',total_comms[total_comms['num_comments']==total_comms['num_comments'].max()])


print('\nWhich day of the week has the most submissions? :')

How many submissions were you able to gather?:
 3100

Who has the most submissions?:
 mkgandkembafan    75
Name: author, dtype: int64

Which submission has the highest score?:
                                                    title  score
1700  Stan Van Gundy shares his thoughts on Orlando ...  31560

Which submission has the highest number of comments?:
                                                    title  num_comments
2900  GAME THREAD: Utah Jazz (41-23) @ New Orleans P...          8991

Which day of the week has the most submissions? :


In [ ]:

idx = pd.date_range(start='2020-07-01', freq='D', periods=30)
idx
#DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03'],
# #             dtype='datetime64[ns]', freq='D')
idx.day_name()
#Index(['Monday', 'Tuesday', 'Wednesday'], dtype='object')